# Exploration

Exploratory work for solving the problem of (1) a more efficient algorithm for discovering dependencies that (2) handles the cases of "ties," or dependencies that are invarient to one another.

## Let's start with (1), the problem with the existing approach

Here's an example:

In [1]:
# Example dictionary of items we want to resolve dependencies for
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D']
}


def dependencies_exist(my_items):
    all_dependencies_exist = True
    for item, dependencies in my_items.items():
        for dependency in dependencies:
            if dependency not in my_items.keys():
                print('Non-existant dependency: ({0}, {1})').format(
                    item, dependency)
                all_dependencies_exist = False
    return all_dependencies_exist


def list_dependencies(my_items, item, verbose=False):
    """Find all dependencies for [item] within [my_items].
    """
    dependencies, new_dependencies_count = list(my_items[item]), -1
    if verbose:
        print("> Indexing:", item)
    num_indent = 2
    while new_dependencies_count != 0:
        new_dependencies_count = 0
        for item in dependencies:
            new_dependencies = [new_item for new_item in my_items[item] 
                                if new_item not in dependencies]
            if verbose:
                print("{} Indexing:".format(">" * num_indent), item)
            dependencies += new_dependencies
            new_dependencies_count += len(new_dependencies)
        num_indent += 1
    return dependencies


def no_circular_dependencies(my_items):
    """Check for any circular dependencies in [my_items].
    """
    not_circular = True
    for item in my_items.keys():
        dependencies = list_dependencies(my_items, item)
        if item in dependencies:
            not_circular = False
            print('Circular dependency for', item)
    return not_circular



print(dependencies_exist(my_items))
print(no_circular_dependencies(my_items))

True
True


The problem is that we're repeating many of the computations many, many times!

In [2]:
def order_dependencies(my_items, verbose=False):
    """Order the dependencies in [my_items], returning a list of keys from
    [my_items] in order such that all dependencies resolve.
    """
    items, index = list(my_items.keys()), 0
    while (len(items) - 1) != index:
        item = items[index]
        if verbose:
            print("Identifying order for:", item)
        current_dependencies = list_dependencies(my_items, item, verbose=verbose)
        no_order_change = True
        for dependency in current_dependencies:
            if items.index(dependency) - items.index(item) > 0:
                items += [items.pop(index)]
                no_order_change = False
                break
        if no_order_change:
            index += 1
    return items


order_dependencies(my_items, verbose=True)

Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>> Indexing: E
>> Indexing: F
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
>>> Indexing: E
>>> Indexing: F
Identifying order for: B
> Indexing: B
Identifying order for: C
> Indexing: C
>> Indexing: D
>> Indexing: B
>> Indexing: E
>> Indexing: F
>>> Indexing: D
>>> Indexing: B
>>> Indexing: E
>>> Indexing: F
Identifying order for: D
> Indexing: D
>> Indexing: B
>> Indexing: E
>> Indexing: F
>>> Indexing: B
>>> Indexing: E
>>> Indexing: F
Identifying order for: E
> Indexing: E
>> Indexing: F
Identifying order for: F
> Indexing: F
Identifying order for: Z
> Indexing: Z
>> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>> Indexing: E
>> Indexing: F
>>> Indexing: A
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
>>> Indexing: E
>>> Indexing: F
Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>> Indexing: E
>> Indexing: F
>>> Indexing: B
>>> Indexing: C
>>> Inde

['B', 'F', 'E', 'D', 'C', 'A', 'Z']

Here's a clearer example, where the problem is truly highlighted:

In [3]:
my_items = {
    "A": ["B"],
    "B": ["C"],
    "C": ["D"],
    "D": []
}


order_dependencies(my_items, verbose=True)

Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
Identifying order for: B
> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: C
>>> Indexing: D
Identifying order for: C
> Indexing: C
>> Indexing: D
Identifying order for: D
> Indexing: D
Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
Identifying order for: B
> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: C
>>> Indexing: D
Identifying order for: C
> Indexing: C
>> Indexing: D
Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
Identifying order for: B
> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: C
>>> Indexing: D


['D', 'C', 'B', 'A']

### Solution for (1) using recursion

The process for just listing the dependencies should also give us the order, these don't need to be separate steps, perhaps except for getting the ideal ordering.

In [4]:
class CircularDependencyException(Exception):
    pass


def enhanced_list_dependencies(my_items, item, known_dependencies={}, call_stack_order=set(), debug=False):
    """List the complete set of items that are dependent on a given item (item) in an 
    items dictionary (my_items).
    
    Parameters
    ----------
    my_items : dict
        A dictionary of {item: list of items that this item depends on}
        
    item : int or str
        The item in my_items that we want to return a full list of items that this item
        depends on
        
    known_dependencies : dict (default of an empty dictionary)
        A dictionary of {item: list of items that this item depends on} that is known to
        be complete. The difference between this and my_items is that, for example, if 
        A is dependent on B which is dependent on C which is dependent on nothing, 
        my_items might look like:
        
        {
            "A": "B",
            "B": "C",
            "C": []
        }
        
        whereas known_dependencies, when it is complete, will look like:
        
        {
            "A": ["B", "C"],
            "B": ["C"],
            "C": []
        }
        
        This function will use known_dependencies as a cache to store known complete
        dependencies. 
        
    call_stack_order : set
        Whenever we have NOT cached the dependencies for an item, we recursively call 
        this function to find that item's dependencies. When this happens, we add the 
        item that we are recursively calling this function for to the call_stack_order. 
        Each item should only be appearing once in this list because we cache (basically
        memoize) the results of each recursive call. The reason we keep this list is to
        discover circular dependencies, aka when item 1 is dependent on item 2 which is
        dependent on item 1, and so forth. If we don't save call_stack_order and use it
        to discover circular dependencies, circular dependencies will result in an infinite
        loop of recursive calls, and will raise an exception when Python's recurssion
        limit is reached.
        
    debug : bool (default of False)
        In debug mode, this function will print out statements as it recursively travels
        the tree of dependencies. 
    
    Returns
    -------
    item_dependencies : list
        The complete list of items that [item] depends on in [my_items]
    
    known_dependencies : dict
        See above, primarily used as a caching mechanism. If the goal here was just to 
        list the complete set of items that an item depends on, we could probably just
        add memoization to this function, but the benefit of using known_dependencies
        is that we can re-use it when looping through an entire items dictionary
    
    """
    
    # List of dependnecies for an item, initially populate with the known dependencies
    item_dependencies = my_items[item]
    if not isinstance(item_dependencies, list):
        item_dependencies = list(item_dependencies)  # -- force everything into lists
        
    if debug:
        print("* Current call stack order:", call_stack_order)
        print("Initial item dependencies for {}: ".format(item), item_dependencies)
        print("Current state of known dependencies:", known_dependencies)
        
    
    # Traverse the tree of dependencies
    for dependency in item_dependencies:
        
            
        # Check to see if we've already cached the dependency or not...
        new_dependencies = known_dependencies.get(dependency)
        
        if debug:
            print("> Looking into dependency: ", dependency)
            print("** New deps:", new_dependencies)
            
        if new_dependencies is not None:
            
            # We HAVE cached the dependencies
            if debug:
                print(">> Dependency for {} known: ".format(dependency), new_dependencies)
           
        else:
            
            # We have NOT cached the dependencies
            if debug:
                print(">> Dependency for {} unknown, initiating recursive call ..... ".format(dependency))
            
            if dependency in call_stack_order:
                raise CircularDependencyException("Circular dependency with item: {}".format(dependency))
            else:
                call_stack_order.add(dependency)
            
            # Rececursively call this function until we known all of the possible dependencies
            new_dependencies, known_dependencies, call_stack_order = enhanced_list_dependencies(
                my_items=my_items, 
                item=dependency, 
                known_dependencies=known_dependencies,
                call_stack_order=call_stack_order,
                debug=debug
            ) 
        
        if new_dependencies:
                
            # Merge the existing and newly discovered dependencies
            item_dependencies = list(set(item_dependencies + new_dependencies))
            
    # When we have all of an item's dependencies, add them to the cache, and return the list of
    # the item's dependencies
    if debug:
        print("& All dependencies for {} are known :) they are: ".format(item), item_dependencies)
    known_dependencies[item] = item_dependencies
    return item_dependencies, known_dependencies, call_stack_order
                          

my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D']
}

enhanced_list_dependencies(my_items, "Z", known_dependencies={}, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for Z:  ['A', 'B', 'C', 'D']
Current state of known dependencies: {}
> Looking into dependency:  A
** New deps: None
>> Dependency for A unknown, initiating recursive call ..... 
* Current call stack order: {'A'}
Initial item dependencies for A:  ['B', 'C', 'D']
Current state of known dependencies: {}
> Looking into dependency:  B
** New deps: None
>> Dependency for B unknown, initiating recursive call ..... 
* Current call stack order: {'B', 'A'}
Initial item dependencies for B:  []
Current state of known dependencies: {}
& All dependencies for B are known :) they are:  []
> Looking into dependency:  C
** New deps: None
>> Dependency for C unknown, initiating recursive call ..... 
* Current call stack order: {'B', 'A', 'C'}
Initial item dependencies for C:  ['D']
Current state of known dependencies: {'B': []}
> Looking into dependency:  D
** New deps: None
>> Dependency for D unknown, initiating recursive call ..... 
* Curren

(['A', 'E', 'D', 'F', 'C', 'B'],
 {'B': [],
  'F': [],
  'E': ['F'],
  'D': ['E', 'F', 'B'],
  'C': ['E', 'D', 'B', 'F'],
  'A': ['E', 'D', 'F', 'C', 'B'],
  'Z': ['A', 'E', 'D', 'F', 'C', 'B']},
 {'A', 'B', 'C', 'D', 'E', 'F'})

In [5]:
my_items = {
    "A": ["B"],
    "B": ["C"],
    "C": ["D"],
    "D": []
}
        
        
enhanced_list_dependencies(my_items, "A", known_dependencies={}, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for A:  ['B']
Current state of known dependencies: {}
> Looking into dependency:  B
** New deps: None
>> Dependency for B unknown, initiating recursive call ..... 
* Current call stack order: {'B'}
Initial item dependencies for B:  ['C']
Current state of known dependencies: {}
> Looking into dependency:  C
** New deps: None
>> Dependency for C unknown, initiating recursive call ..... 
* Current call stack order: {'B', 'C'}
Initial item dependencies for C:  ['D']
Current state of known dependencies: {}
> Looking into dependency:  D
** New deps: None
>> Dependency for D unknown, initiating recursive call ..... 
* Current call stack order: {'D', 'B', 'C'}
Initial item dependencies for D:  []
Current state of known dependencies: {}
& All dependencies for D are known :) they are:  []
& All dependencies for C are known :) they are:  ['D']
& All dependencies for B are known :) they are:  ['D', 'C']
& All dependencies for A are known 

(['D', 'B', 'C'],
 {'D': [], 'C': ['D'], 'B': ['D', 'C'], 'A': ['D', 'B', 'C']},
 {'B', 'C', 'D'})

In [6]:
def dependencies_exist(my_items, verbose=True):
    all_dependencies_exist = True
    possible_dependencies = list(my_items.keys())
    for item, dependencies in my_items.items():
        for dependency in dependencies:
            if dependency not in possible_dependencies:
                if verbose:
                    print("Non-existant dependency: ({0}, {1})".format(item, dependency))
                all_dependencies_exist = False
    return all_dependencies_exist


# Example dictionary of items we want to resolve dependencies for
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['A'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D', 'Y']
}

print("All dependencies exist:", dependencies_exist(my_items))
print("Let's fix this...but add in a circular dependency")
my_items['Z'] = ['A', 'B', 'C', 'D']

## PROBLEM: THE EXISTING enhanced_list_dependencies FUNCTION FAILS FOR CIRCULAR DEPENDENCIES! WE NEED TO FIX THIS!!!
results = enhanced_list_dependencies(my_items, "Z", known_dependencies={}, call_stack_order=set(), debug=True)

Non-existant dependency: (Z, Y)
All dependencies exist: False
Let's fix this...but add in a circular dependency
* Current call stack order: set()
Initial item dependencies for Z:  ['A', 'B', 'C', 'D']
Current state of known dependencies: {}
> Looking into dependency:  A
** New deps: None
>> Dependency for A unknown, initiating recursive call ..... 
* Current call stack order: {'A'}
Initial item dependencies for A:  ['B', 'C', 'D']
Current state of known dependencies: {}
> Looking into dependency:  B
** New deps: None
>> Dependency for B unknown, initiating recursive call ..... 
* Current call stack order: {'B', 'A'}
Initial item dependencies for B:  []
Current state of known dependencies: {}
& All dependencies for B are known :) they are:  []
> Looking into dependency:  C
** New deps: None
>> Dependency for C unknown, initiating recursive call ..... 
* Current call stack order: {'B', 'A', 'C'}
Initial item dependencies for C:  ['D']
Current state of known dependencies: {'B': []}
> Look

CircularDependencyException: Circular dependency with item: A

Notice that the order of keys in the `known_dependencies` dict returned by `enhanced_list_dependencies` is giving us the same order as `order_dependencies` :)

--------------------

Now, let's try saving `known_dependencies` and looping through all of the keys in `my_items` to find the dependencies for each, starting with `B`

In [7]:
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D']
}

deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "B", known_dependencies={}, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for B:  []
Current state of known dependencies: {}
& All dependencies for B are known :) they are:  []


Now let's get `D`, which should already know the result of `B`

In [8]:
deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "D", known_dependencies=known_dependencies, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for D:  ['B', 'E']
Current state of known dependencies: {'B': []}
> Looking into dependency:  B
** New deps: []
>> Dependency for B known:  []
> Looking into dependency:  E
** New deps: None
>> Dependency for E unknown, initiating recursive call ..... 
* Current call stack order: {'E'}
Initial item dependencies for E:  ['F']
Current state of known dependencies: {'B': []}
> Looking into dependency:  F
** New deps: None
>> Dependency for F unknown, initiating recursive call ..... 
* Current call stack order: {'E', 'F'}
Initial item dependencies for F:  []
Current state of known dependencies: {'B': []}
& All dependencies for F are known :) they are:  []
& All dependencies for E are known :) they are:  ['F']
& All dependencies for D are known :) they are:  ['E', 'F', 'B']


And `E` and `F`, which should already be known

In [9]:
deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "E", known_dependencies=known_dependencies, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for E:  ['F']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['E', 'F', 'B']}
> Looking into dependency:  F
** New deps: []
>> Dependency for F known:  []
& All dependencies for E are known :) they are:  ['F']


In [10]:
deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "F", known_dependencies=known_dependencies, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for F:  []
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['E', 'F', 'B']}
& All dependencies for F are known :) they are:  []


And `C`, which should only call `D` which is already known

In [11]:
deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "C", known_dependencies=known_dependencies, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for C:  ['D']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['E', 'F', 'B']}
> Looking into dependency:  D
** New deps: ['E', 'F', 'B']
>> Dependency for D known:  ['E', 'F', 'B']
& All dependencies for C are known :) they are:  ['E', 'D', 'B', 'F']


And `A`, which should call `B`, `C`, and `D` which are all already known

In [12]:
deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "A", known_dependencies=known_dependencies, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for A:  ['B', 'C', 'D']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['E', 'F', 'B'], 'C': ['E', 'D', 'B', 'F']}
> Looking into dependency:  B
** New deps: []
>> Dependency for B known:  []
> Looking into dependency:  C
** New deps: ['E', 'D', 'B', 'F']
>> Dependency for C known:  ['E', 'D', 'B', 'F']
> Looking into dependency:  D
** New deps: ['E', 'F', 'B']
>> Dependency for D known:  ['E', 'F', 'B']
& All dependencies for A are known :) they are:  ['E', 'D', 'F', 'C', 'B']


And, finally, `Z`

In [13]:
deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
    my_items, "Z", known_dependencies=known_dependencies, call_stack_order=set(), debug=True)

* Current call stack order: set()
Initial item dependencies for Z:  ['A', 'B', 'C', 'D']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['E', 'F', 'B'], 'C': ['E', 'D', 'B', 'F'], 'A': ['E', 'D', 'F', 'C', 'B']}
> Looking into dependency:  A
** New deps: ['E', 'D', 'F', 'C', 'B']
>> Dependency for A known:  ['E', 'D', 'F', 'C', 'B']
> Looking into dependency:  B
** New deps: []
>> Dependency for B known:  []
> Looking into dependency:  C
** New deps: ['E', 'D', 'B', 'F']
>> Dependency for C known:  ['E', 'D', 'B', 'F']
> Looking into dependency:  D
** New deps: ['E', 'F', 'B']
>> Dependency for D known:  ['E', 'F', 'B']
& All dependencies for Z are known :) they are:  ['A', 'E', 'D', 'F', 'C', 'B']


#### Comparing the new and old algorithms

In [14]:
def extreme_dependency_generator(dependency_depth):
    """Generate a list of items where each item is dependent on the subsequent item, aka an 
    extreme case of dependencies.
    """
    items_with_dependencies = {}
    for number in range(1, dependency_depth):
        items_with_dependencies[number] = [number + 1]
    items_with_dependencies[dependency_depth] = []
    return items_with_dependencies

In [15]:
%%time

results = enhanced_list_dependencies(extreme_dependency_generator(100), 1, known_dependencies={}, 
                                     call_stack_order=set(), debug=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 672 µs


How does the running time for the recursion depth of 100 compare with the old algorithm?

* ANSWER: No faster :O

In [16]:
%%time 

results = list_dependencies(extreme_dependency_generator(100), 1, verbose=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 226 µs


How far can we push this before hitting recursion limits?

In [17]:
%%time

results = enhanced_list_dependencies(extreme_dependency_generator(999), 1, known_dependencies={}, 
                                     call_stack_order=set(), debug=False)

CPU times: user 15.6 ms, sys: 15.6 ms, total: 31.2 ms
Wall time: 34.4 ms


Versus the old algorithm?

* WELL...not exactly a fair test if we're looping through all of the items, right?

In [18]:
%%time 

results = list_dependencies(extreme_dependency_generator(999), 1, verbose=False)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 13.5 ms


OK, let's do a real test where we're iterating over ALL of the items and finding each one's dependencies, which is what the old ordering did




In [19]:
my_items = extreme_dependency_generator(500)

In [20]:
%%time

# OLD ALGORITHM

for item in my_items:
    item_dependencies = list_dependencies(my_items, item, verbose=False)

CPU times: user 578 ms, sys: 15.6 ms, total: 594 ms
Wall time: 567 ms


In [21]:
%%time

# OLD ALGORITHM for ordering

results = order_dependencies(my_items)

CPU times: user 3min 29s, sys: 31.2 ms, total: 3min 29s
Wall time: 3min 30s


In [23]:
%%time

# NEW ALGORITHM

known_dependencies = {}
for item in my_items:
    item_dependencies, known_dependencies, call_stack_order  = enhanced_list_dependencies(
        my_items, item, known_dependencies, call_stack_order=set(), debug=False)

CPU times: user 0 ns, sys: 15.6 ms, total: 15.6 ms
Wall time: 17.6 ms


#### Finding the recursion limit

* Getting rid of the cell output because it's taking up too much space, but 

```
results = enhanced_list_dependencies(extreme_dependency_generator(5000), 1, known_dependencies={}, debug=False)
```

Results in a 

```
RecursionError: maximum recursion depth exceeded in comparison
```

Can we get around the recursion limit?

In [24]:
import sys
print("Current recursion limit:", sys.getrecursionlimit())
sys.setrecursionlimit(10000)
print("Changing recursion limit to:", sys.getrecursionlimit())

Current recursion limit: 3000
Changing recursion limit to: 10000


In [25]:
%%time

results = enhanced_list_dependencies(extreme_dependency_generator(5000), 1, 
                                     known_dependencies={}, call_stack_order=set(), debug=False)

CPU times: user 422 ms, sys: 281 ms, total: 703 ms
Wall time: 702 ms


------------

This algorithm is looking good...but how can we make it iterative instead of recursive? Personally, I like the elegance of the recursive solution, and it makes most sense to me, but Python isn't optimized for tail call recursion, and has a default recursion depth of 1,000, which is fairly low.

### Solution for (1) using iteration

My original solution used iteration, but in a bad way because computations were repeated (imagine computing the Fibonacci sequence where each step has to be recomputed for each subsequent step...not good).

REVISIT LATER!!!!

--------------------

## All possible orderings

There can be multiple correct orderings of dependencies...how do we find this out?

In [86]:
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D'],
}

known_dependencies = {}
for item in my_items:
    print("Finding deps for:", item)
    item_deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
        my_items, item, known_dependencies=known_dependencies, call_stack_order=set(), debug=False)
    
known_dependencies

Finding deps for: A
Finding deps for: B
Finding deps for: C
Finding deps for: D
Finding deps for: E
Finding deps for: F
Finding deps for: Z


{'B': [],
 'F': [],
 'E': ['F'],
 'D': ['E', 'F', 'B'],
 'C': ['E', 'D', 'B', 'F'],
 'A': ['E', 'D', 'F', 'C', 'B'],
 'Z': ['A', 'E', 'D', 'F', 'C', 'B']}

Does the ordering of the keys in `known_dependencies` stay the same if we change the order of the keys?

* NO, this will give us ALL POSSIBLE orderings...

In [87]:
from itertools import permutations 


# Loop through all permutations of my_items, generate a known_dependencies dict
known_deps_dicts = []
for my_items_perm in list(permutations(my_items.keys())):
    known_dependencies = {}
    for item in list(my_items_perm):
        item_deps, known_dependencies, call_stack_order = enhanced_list_dependencies(
            my_items, item, known_dependencies=known_dependencies, call_stack_order=set(), debug=False)
    known_deps_dicts.append(known_dependencies)
    
    
# Do all of the known_dependencies dicts have keys in the same order?
# NO, permutating through all possible orderings of the keys in my_items is GIVING us all
# possible orderings :)
known_deps_dicts_keys = [tuple(x.keys()) for x in known_deps_dicts]
set(known_deps_dicts_keys)

{('B', 'F', 'E', 'D', 'C', 'A', 'Z'),
 ('F', 'B', 'E', 'D', 'C', 'A', 'Z'),
 ('F', 'E', 'B', 'D', 'C', 'A', 'Z')}

In [90]:
[list(x) for x in set(known_deps_dicts_keys)]

[['F', 'E', 'B', 'D', 'C', 'A', 'Z'],
 ['F', 'B', 'E', 'D', 'C', 'A', 'Z'],
 ['B', 'F', 'E', 'D', 'C', 'A', 'Z']]

Let's write a function to check whether or not an ordering is correct (a naive algorithm for finding all possible correct orderings would just be to permutate over all possible orderings, and check whether or not each one is correct)

In [88]:
def check_if_ordering_is_correct(ordering, known_dependencies):
    """Given an [ordering] of items and a complete dictionary of items to their 
    dependencies [known_dependencies], check to see if the ordering is correct from 
    a dependency management perspective or not.
    """
    items_already_looped_through = set()
    for item in ordering:
        items_already_looped_through.add(item)
        # Loop through each item's complete set of dependencies, if any of those dependencies
        # haven't already been looped through, then the ordering is incorrect!
        for item_dependency in known_dependencies[item]:
            if item_dependency not in items_already_looped_through:
                return False
    return True

for ordering in set(known_deps_dicts_keys):
    print("The ordering is correct:", check_if_ordering_is_correct(ordering, known_dependencies))

The ordering is correct: True
The ordering is correct: True
The ordering is correct: True


In [89]:
incorrect_orderings = []
correct_orderings = []
for my_items_perm in list(permutations(my_items.keys())):
    if check_if_ordering_is_correct(my_items_perm, known_dependencies):
        correct_orderings.append(my_items_perm)
    else:
        incorrect_orderings.append(my_items_perm)
        
print("Number of permutations:", len(incorrect_orderings) + len(correct_orderings))
print("Number of correct orderings:", len(correct_orderings))
print("Number of incorrect orderings:", len(incorrect_orderings))

Number of permutations: 5040
Number of correct orderings: 3
Number of incorrect orderings: 5037


Can we programatically figure out the correct orderings w/o having to 

In [82]:
known_dependencies

{'H': [],
 'B': [],
 'F': [],
 'E': ['F'],
 'D': ['E', 'F', 'B'],
 'C': ['E', 'D', 'B', 'F'],
 'A': ['E', 'D', 'F', 'C', 'B'],
 'Z': ['A', 'E', 'D', 'F', 'C', 'B']}

In [83]:
items_that_depend_on_me = {}
for item in known_dependencies.keys():
    items_that_depend_on_me[item] = []
    for sub_item in known_dependencies.keys():
        if item in known_dependencies[sub_item]:
            items_that_depend_on_me[item].append(sub_item)
            

items_that_depend_on_me

{'H': [],
 'B': ['D', 'C', 'A', 'Z'],
 'F': ['E', 'D', 'C', 'A', 'Z'],
 'E': ['D', 'C', 'A', 'Z'],
 'D': ['C', 'A', 'Z'],
 'C': ['A', 'Z'],
 'A': ['Z'],
 'Z': []}

In [84]:
initial_ordering = list(items_that_depend_on_me.keys())
initial_ordering

['H', 'B', 'F', 'E', 'D', 'C', 'A', 'Z']

In [ ]:
def order_items(known_dependencies, items_that_depend_on_me, current_item, current_ordering=[],  valid_orderings=[]):
    """Identify all possible orderings
    """
    possible_items = list(known_dependencies.keys())
    unordered_items = [item for item in possible_items if item not in current_ordering]
    if len(unordered_items) > 0:
        # Not done constructing the valid ordering
        for item in unordered_items:
            if 
    else:
        # Done constructing the ordering, is it valid though?
        if len(current_ordering)
        
        
    
    
current_ordering, valid_orderings = [[] for _ in range(2)]
beginning = [item for item, dependencies in known_dependencies.items() if dependencies == []]
for current_item in beginning:
    current_ordering, valid_orderings = order_items(
        known_dependencies, items_that_depend_on_me, current_item, 
        current_ordering=current_ordering, 
        valid_orderings=valid_orderings
    )